In [2]:
import cv2 
import numpy as np 
import time 

# replace the red pixels ( or undesired area ) with 
# background pixels to generate the invisibility feature. 
## 1. Hue: This channel encodes color information. Hue can be 
# thought of an angle where 0 degree corresponds to the red color, 
# 120 degrees corresponds to the green color, and 240 degrees 
# corresponds to the blue color. 
## 2. Saturation: This channel encodes the intensity/purity of color. 
# For example, pink is less saturated than red. 
## 3. Value: This channel encodes the brightness of color. 

In [21]:
# using a webcam
cam = cv2.VideoCapture(1)
out = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc(*'XVID'), 25, (640,480))
cam.set(3,640)
cam.set(4,480)
cam.set(10,130)
time.sleep(1) 
count = 0
background = 0

# capturing the background in range of 60 you should have video that have some seconds 
# dedicated to background frame so that it could easily save the background image 
for i in range(60): 
    return_val, background = cam.read() 
    if return_val == False : 
        continue

background = np.flip(background, axis = 1) # flipping of the frame 

# we are reading from video 
while (cam.isOpened()): 
    return_val, img = cam.read() 
    if not return_val : 
        break
    count = count + 1
    img = np.flip(img, axis = 1) 

    # convert the image - BGR to HSV 
    # as we focused on detection of red color 

    # converting BGR to HSV for better detection
    
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) 

    
    # ranges should be carefully chosen setting the lower and upper range for mask1 
    lower_blue = np.array([72,50,100])
    upper_blue = np.array([130,255,255])
    mask1 = cv2.inRange(hsv, lower_blue, upper_blue) 
 
    mask1 = cv2.morphologyEx(mask1, cv2.MORPH_OPEN, np.ones((3, 3), 
                                        np.uint8), iterations = 2) 
    mask1 = cv2.dilate(mask1, np.ones((3, 3), np.uint8), iterations = 1) 
    mask2 = cv2.bitwise_not(mask1) 

    # Generating the final output 
    res1 = cv2.bitwise_and(background, background, mask = mask1) 
    res2 = cv2.bitwise_and(img, img, mask = mask2) 
    final_output = cv2.addWeighted(res1, 1, res2, 1, 0) 
    out.write(final_output.copy())
    cv2.imshow("INVISIBLE MAN", final_output)
    if cv2.waitKey(1) & 0xFF == ord('a'):
        break
cam.release()
out.release()
cv2.destroyAllWindows()